In [1]:
import os
import sys
sys.path.append('../')

import numpy as np
import matplotlib
from matplotlib.dates import DateFormatter
import matplotlib.pyplot as plt
import datetime
import pandas as pd
from pprint import pprint
from scipy.optimize import curve_fit

import src.io as sio
import src.preprocessing as spp
import src.fitting as sft
import ipympl
import pytz

In [2]:
BAKEOUT_FOLDER = sio.get_folderpath("20201030_Bakeout")

# Extrapolation

In [ ]:
df1 = sio.read_tm224_data("temperature-monitoring04.xls", BAKEOUT_FOLDER)
df2 = sio.read_tm224_data("temperature-monitoring05.xls", BAKEOUT_FOLDER)
df3 = sio.read_tm224_data("temperature-monitoring06.xls", BAKEOUT_FOLDER)

df = pd.concat([df1, df2, df3])

fig, ax = plt.subplots()

myFmt = DateFormatter("%H:%M\n%a")
ax.xaxis.set_major_formatter(myFmt)
ax.plot(df["MPL_datetimes"], df["Baseplate"])

x, y, x_mod, model = sft.time_extrapolation_lmfit(df, "Baseplate", end_date="11-Nov-20 9:00", start_index=0, fit="linear")
ax.plot(df["MPL_datetimes"], model.best_fit)

In [ ]:
df = sio.read_tm224_data("temperature-monitoring07.xls", BAKEOUT_FOLDER)

fig, ax = plt.subplots()

myFmt = DateFormatter("%H:%M\n%a")
ax.xaxis.set_major_formatter(myFmt)
ax.plot(df["MPL_datetimes"], df["Baseplate"], color="C2")

x, y, x_mod, model = sft.time_extrapolation_lmfit(df, "Baseplate", end_date="11-Nov-20 12:00", fit="quadratic")
#ax.plot(x_mod, model.best_fit)
ax.plot(x, y, "-", color="C1")

In [ ]:
df = sio.read_tm224_data("temperature-monitoring07.xls", BAKEOUT_FOLDER)

fig, ax = plt.subplots()

myFmt = DateFormatter("%H:%M\n%a")
ax.xaxis.set_major_formatter(myFmt)
ax.plot(df["MPL_datetimes"], df["Baseplate"], color="C2")

x, y = sft.time_extrapolation(df, "Baseplate", end_date="11-Nov-20 12:00", fit="linear")
#ax.plot(x_mod, model.best_fit)
ax.plot(x, y, "-", color="C1")

# Pressure

In [ ]:
df1 = sio.read_tpg_data("pressure-monitoring01", BAKEOUT_FOLDER)
df2 = sio.read_tpg_data("pressure-monitoring02", BAKEOUT_FOLDER)
df3 = sio.read_tpg_data("pressure-monitoring03", BAKEOUT_FOLDER)

df = pd.concat([df1, df2, df3])

fig, ax = plt.subplots()

myFmt = DateFormatter("%H:%M\n%a")
ax.xaxis.set_major_formatter(myFmt)
ax.set_yscale("log")

ax.set_ylabel(r"Pressure (mbar)")

ax.plot(df["MPL_datetimes"], df["Prep"], ".", label="Main")
ax.legend()

[]


SyntaxError: 'break' outside loop (<ipython-input-15-72f52ba15f95>, line 10)

# Increasing controllers to 80%

In [ ]:
df1 = sio.read_tm224_data("temperature-monitoring08.xls", BAKEOUT_FOLDER)
df2 = sio.read_tm224_data("temperature-monitoring09.xls", BAKEOUT_FOLDER)

df = pd.concat([df1, df2])
fig, ax = plt.subplots()

a = 4500

myFmt = DateFormatter("%H:%M\n%a")
ax.xaxis.set_major_formatter(myFmt)
ax.plot(df["MPL_datetimes"][a:], df["Baseplate"][a:], ".", color="C2")

x, y, x_mod, model = sft.time_extrapolation_lmfit(df, "Baseplate", start_index=5500, end_date="12-Nov-20 12:00", fit="linear")
    
print(sft.setpointy_reach_time(x, y, setpointy=377.6))
print(model.fit_report())

ax.plot(x, y, "--", color="C1")

# Combined pressure and temperature

In [3]:
# Pressure
df1 = sio.read_tpg_data("pressure-monitoring01", BAKEOUT_FOLDER)
df2 = sio.read_tpg_data("pressure-monitoring02", BAKEOUT_FOLDER)
df3 = sio.read_tpg_data("pressure-monitoring03", BAKEOUT_FOLDER)
df4 = sio.read_tpg_data("pressure-monitoring04", BAKEOUT_FOLDER)
dfp = pd.concat([df1, df2, df3, df4])

# Get rid of error value recorded when restarting gauge
dfp = dfp.drop(dfp.index[dfp['Prep'] == 0].tolist())


# Temperature
df1 = sio.read_tm224_data("temperature-monitoring01.xls", BAKEOUT_FOLDER)
df2 = sio.read_tm224_data("temperature-monitoring02.xls", BAKEOUT_FOLDER)
df3 = sio.read_tm224_data("temperature-monitoring03.xls", BAKEOUT_FOLDER)
df4 = sio.read_tm224_data("temperature-monitoring04.xls", BAKEOUT_FOLDER)
df5 = sio.read_tm224_data("temperature-monitoring05.xls", BAKEOUT_FOLDER)
df6 = sio.read_tm224_data("temperature-monitoring06.xls", BAKEOUT_FOLDER)
df7 = sio.read_tm224_data("temperature-monitoring07.xls", BAKEOUT_FOLDER)
df8 = sio.read_tm224_data("temperature-monitoring08.xls", BAKEOUT_FOLDER)
df9 = sio.read_tm224_data("temperature-monitoring09.xls", BAKEOUT_FOLDER)
df10 = sio.read_tm224_data("temperature-monitoring10.xls", BAKEOUT_FOLDER)
df11 = sio.read_tm224_data("temperature-monitoring11.xls", BAKEOUT_FOLDER)
dft = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11])

In [8]:
fig, ax = plt.subplots()

myFmt = DateFormatter("%a\n%d-%m")
ax.xaxis.set_major_formatter(myFmt)

# Pressure
ax.set_ylabel(r"Pressure (mbar)", color="C0")
ax.set_yscale("log", base=10)
ax.tick_params(axis='y', labelcolor="C0")
ax.plot(dfp["MPL_datetimes"], dfp["Prep"], "-", color="C0")

x, y, = sft.time_extrapolation(dfp, "Prep", end_date="16-Nov-20 09:00", start_index=105000, fit="offset_exponentional")
ax.plot(x, y, "--")

ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis
ax2.set_ylabel(r"Baseplate Temperature (K)", color="C1")
ax2.plot(dft["MPL_datetimes"], dft["Baseplate"], "-", color="C1")

x, y, x_mod, model = sft.time_extrapolation_lmfit(dft, "Baseplate", end_date="16-Nov-20 09:00", start_index=65000)
ax2.plot(x, y, "--")
ax2.tick_params(axis='y', labelcolor="C1")

#sio.savefig("pressure-monitoring01-04-temperature-monitoring01-11", BAKEOUT_FOLDER)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …